In [59]:
import torch
import deepwave
import numpy as np
from omegaconf import OmegaConf
from pyexpat import features

import matplotlib.pyplot as plt
from soupsieve.util import lower
from utils import *
from einops import rearrange
from omegaconf import OmegaConf
from utils.utils_plot import plot_model,plot_shot
import  torch
import  numpy as np
from utils import make_and_load_model

# 设置config文件路径

In [60]:
config_path = "../config/template_he.yaml"
cfg = OmegaConf.load(config_path)

# 设置loc

In [61]:
dx = 50
n_shots = 140
src_depth = 50 / 50
first_src =  200 / 50
d_src = 125 / 50

n_recs = 474
rec_depth = 6 / 50
first_rec = 12.5 / 50
d_rcv = 37.5 / 50
#============================================================================
src_loc = np.zeros([n_shots,1,2])
src_loc[:,0,0] = np.arange(n_shots) * d_src + first_src
src_loc[:,0,1] = src_depth
rec_loc = np.zeros([n_shots,n_recs,2])
rec_loc[:,:,0] = np.tile((np.arange(n_recs)*d_rcv+first_rec),(n_shots,1))
rec_loc[:,:,1]=rec_depth
src_loc_path = cfg.forward_par.src_loc_path
rec_loc_path = cfg.forward_par.rec_loc_path
np.save(src_loc_path,src_loc)
np.save(rec_loc_path,rec_loc)
src_loc = torch.from_numpy(src_loc).float()
rec_loc = torch.from_numpy(rec_loc).float()

# 设置地震子波

In [62]:
with open('/data/yibiao/nn_loss/experiments/he/st.1.5-4.0', 'rb') as f:
    wavelet = np.fromfile(f, dtype=np.float32).reshape(1,1,-1)
wavelet = wavelet.repeat(n_shots,axis=0)
wavelet_path = cfg.forward_par.wavelet_path
np.save(wavelet_path,wavelet)
wavelet = torch.from_numpy(wavelet).float()

# 设置初始模型以及观测数据

In [63]:
vp_true = np.fromfile('/data/yibiao/nn_loss/experiments/he/vp.true', dtype=np.float32).reshape(nx, nz)
vp_init = np.fromfile('/data/yibiao/nn_loss/experiments/he/vp.zero', dtype=np.float32).reshape(nx, nz)

obs_signal = deepwave.scalar(torch.from_numpy(vp_true).float(), source_amplitudes=wavelet,source_locations=src_loc,receiver_locations=rec_loc,grid_spacing=cfg.forward_par.grid_spacing,dt=cfg.forward_par.dt,max_vel=cfg.forward_par.max_vel,pml_width=OmegaConf.to_object(cfg.forward_par.pml_width))[-1]

obs_signal_path = cfg.generate_par.obs_signal_path
vp_true_path = cfg.forward_par.vp_true_path
vp_init_path =  cfg.forward_par.vp_init_path

np.save(obs_signal_path,obs_signal)
np.save(vp_init_path,vp_init)
np.save(vp_true_path,vp_true)

